In [1]:
import json
from tqdm import tqdm
import requests

In [11]:
!pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/643.9 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/643.9 kB ? eta -:--:--
   -------------------------------- ------- 524.3/643.9 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 643.9/643.9 kB 1.6 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [14]:
import os
from openai import OpenAI

# os.environ["OPENAI_API_KEY"] = ""  # your actual key
openai_client = OpenAI()

In [15]:
def model(prompt):
    try:
        completion = openai_client.chat.completions.create(
            model="gpt-4o-mini",  # or "gpt-3.5-turbo" / "gpt-4" based on needs
            messages=[
                {"role": "system", "content": "You are a helpful assistant that generates questions and answers based on passages from historical texts."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.3,
        )
        return completion.choices[0].message.content.strip()

    except Exception as e:
        print(f"OpenAI API Error: {e}")
        return None

In [16]:
# Load Data
with open('./data-collection/data/chapter-data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
# url = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-3B-Instruct"
# token = ""  # Replace with your Hugging Face token

# def model(prompt):
#     headers = {
#       'Authorization': f'Bearer {token}'
#     }

#     parameters = {
#       "max_new_tokens": 100,
#       "temperature": 0.01,
#       "top_k": 50,
#       "top_p": 0.95,
#       "return_full_text": False
#     }

#     payload = {"inputs": prompt, "parameters": parameters}

#     response = requests.post(url, headers=headers, json=payload)
    
#     if response.status_code == 200:
#         print("?", response.json())
#         return response.json()[0]['generated_text']
#     else:
#         print(f'Error: {response.status_code}')
#         print(response.json())
#         return None

In [18]:
dataset = []

# Loop through each excerpt
for entry in tqdm(data):
    book_title = entry.get("book_title", "")
    chapter_name = entry.get("chapter_name", "")
    paragraphs = entry.get("paragraphs", [])

    for paragraph in paragraphs:
        if len(paragraph) < 100:
            continue

        input_data = f"Book Title: {book_title}\nChapter Name: {chapter_name}\n\n{paragraph}"

        # Generate question 
        question_prompt = f"Generate one reflective or thoughtful question based on the following passage:\n\n{paragraph}"

        # response_question = model(question_prompt)
        # question = response_question.choices[0].message.content.strip()
        question = model(question_prompt)

        # Generate answer from question
        answer_prompt = (
            f"Answer the following question in the style of the historical figure who wrote the text below:\n\n"
            f"Text: {paragraph}\n\nQuestion: {question}\n\nAnswer:"
        )

        # response_answer = model(answer_prompt)
        # print(response_answer)
        # answer = response_answer.choices[0].message.content.strip()
        answer = model(answer_prompt)

        dataset.append({
            "question": question,
            "input": input_data,
            "answer": answer
        })

    if len(dataset) >= 10:
        break

  0%|          | 6/2601 [01:01<7:21:15, 10.20s/it]


In [19]:

# Save the formatted instruction-tuning data
with open('instruction_tuning_dataset.jsonl', 'w') as f:
    for item in dataset:
        f.write(json.dumps(item) + "\n")

In [20]:
with open('instruction_tuning_dataset.jsonl', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        item = json.loads(line)
        print(f"\nSample {i+1}")
        print("Input:", item["input"][:200], "...")
        print("Question:", item["question"])
        print("Answer:", item["answer"])



Sample 1
Input: Book Title: BOOK XI. THE VARIOUS KINDS OF INSECTS.
Chapter Name: CHAP. 6. (5.)—THE MEANING OF THE TERMS COMMOSIS, PISSO- CEROS, AND PROPOLlS.

The persons who understand this subject, call the substan ...
Question: What might the use of propolis in honeycombs reveal about the relationship between nature and medicine in ancient practices?
Answer: Verily, the use of propolis in the construction of honeycombs unveils a profound understanding of the harmonious relationship betwixt nature and the art of healing in our ancient practices. This substance, derived from the sweet gum of the vine or poplar, serves not merely as a structural element within the hive, but as a potent agent in the preservation of health. It acts as a barrier against the insidious chill and other deleterious influences that threaten the sanctity of the hive, thus safeguarding the precious honey and the bees themselves.

In our wisdom, we discern that nature bestows upon us remedies and protective meas